In [1]:
import xlwings as xw
import time

# 以我們上一堂課的 Excel 檔案爲例，打開你的 tsmc_back_test.xlsx 檔案
wb = xw.Book(r'C:\Users\user\Desktop\PYTHON_EXCEL_LESSON3\tw_stock_portfolio.xlsx')

wb.sheets 可以搜尋出該工作簿底下所有的試算表，注意它是以 List 的方式回傳：
```python
Sheets([<Sheet [20180915_tw_stock_portfolio.xlsx]TW2882>, <Sheet [20180915_tw_stock_portfolio.xlsx]TW2454>, <Sheet [20180915_tw_stock_portfolio.xlsx]TW2330>])
```

In [12]:
wb.sheets
sheet = wb.sheets["TW2330"]
sheet.activate()
for sheet in wb.sheets:
    # 每迭代到一個試算表，就把它開啓
    sheet.activate()
    # 隔兩秒後，迭代到下一個試算表
    time.sleep(2)

 ```python
sheet = wb.sheets["TW2330"]
# sheet.activate 代表開啓 sheet 所代表的試算表
sheet.activate()
```

接下來我們就可以用一個 for 回圈開啓每一個試算表：

```python
for sheet in wb.sheets:
    # 每迭代到一個試算表，就把它開啓
    sheet.activate()
    # 隔兩秒後，迭代到下一個試算表
    time.sleep(2)
```

In [15]:

from xlwings.constants import Direction

# 定義一個 run_back_test 
def run_back_test(tsmc_sheet):
    # 從 B1 儲存格開始，往下查找到最後一個有值的儲存格
    last_cell = tsmc_sheet.range('A1').end(Direction.xlDown)
    # 截取該儲存格的 row 值
    last_row = last_cell.row

    # 設定我們的範例試算表上的名稱
    tsmc_sheet.range('K2:K11').name = 'weight10d'
    tsmc_sheet.range('K2:K6').name = 'weight5d'


    # 5日加權移動平均計算
    for i in range(6, last_row+1):
        # 由於我們需要把兩個陣列相乘，因此這是一個 Excel 的陣列運算
        formula = "=SUM(B{}:B{}*weight5d)/SUM(weight5d)".format(i-4, i)
        # 若一個 Excel 的公式使用到陣列運算，需要用 .formula_array 設定
        tsmc_sheet.cells(i, 'C').formula_array = formula

    # 10日加權移動平均計算
    for i in range(11, last_row+1):
        # 由於我們需要把兩個陣列相乘，因此這是一個 Excel 的陣列運算
        formula = "=SUM(B{}:B{}*weight10d)/SUM(weight10d)".format(i-9, i)
        # 若一個 Excel 的公式使用到陣列運算，需要用 .formula_array 設定
        tsmc_sheet.cells(i, 'D').formula_array = formula  


    # 計算第一天的交易 2017/10/20
    tsmc_sheet.cells(11, 'E').value = 1000 if tsmc_sheet.cells(11, 'C').value > tsmc_sheet.cells(11, 'D').value else 0
    tsmc_sheet.cells(11, 'F').value = 1000 if tsmc_sheet.cells(11, 'D').value > tsmc_sheet.cells(11, 'C').value else 0
    tsmc_sheet.cells(11, 'G').value = tsmc_sheet.cells(11, 'E').value + tsmc_sheet.cells(11, 'F').value
    tsmc_sheet.cells(11, 'H').value =  tsmc_sheet.cells(18, 'L').value - tsmc_sheet.cells(11, 'B').value * 1000
    tsmc_sheet.cells(11, 'I').value = tsmc_sheet.cells(11, 'H').value +  tsmc_sheet.cells(11, 'B').value * tsmc_sheet.cells(11, 'G').value


    # 實作交易策略
    for i in range(12, last_row+1):
        # 截取當天的 5日加權移動平均
        short_term_ma = tsmc_sheet.cells(i, 'C').value
        # 截取當天的 10日加權移動平均
        long_term_ma = tsmc_sheet.cells(i, 'D').value
        # 截取當天收盤價
        price_today = tsmc_sheet.cells(i, 'B').value
        # 若 5日 > 10日，而且我有足夠買入以今日收盤價計價的 1000 股的現金，就買入 1000 股（在 E 欄顯示 1000）
        if (short_term_ma > long_term_ma) and (tsmc_sheet.cells(i-1, 'H').value > price_today * 1000):
            tsmc_sheet.cells(i, 'E').value = 1000
        else:
        # 若上述條件不符和，就買入 0 股，（在 E 欄顯示 0）
            tsmc_sheet.cells(i, 'E').value = 0
        # 若 10日 > 5日，而且昨天的持有股數大於 1000 股，就賣出 1000 股

        if (long_term_ma > short_term_ma) and (tsmc_sheet.cells(i-1, 'G').value >= 1000):
            tsmc_sheet.cells(i, 'F').value = 1000
        else:
            tsmc_sheet.cells(i, 'F').value = 0
        # 持有股數，算法是前一天的持有股數 + 今天的買入股數 - 今天的賣出股數
        tsmc_sheet.cells(i, 'G').value = tsmc_sheet.cells(i-1, 'G').value + tsmc_sheet.cells(i, 'E').value - tsmc_sheet.cells(i, 'F').value
        # 持有資金，算法是前一天的持有資金 + 今日收盤價 x (今天的賣出股數 - 今天的買入股數)
        tsmc_sheet.cells(i, 'H').value = tsmc_sheet.cells(i-1, 'H').value + price_today * (tsmc_sheet.cells(i, 'F').value - tsmc_sheet.cells(i, 'E').value)
        # 總資產則是持有股數 x 今日收盤價 + 今日持有資金
        tsmc_sheet.cells(i, 'I').value = tsmc_sheet.cells(i, 'H').value + tsmc_sheet.cells(i, 'G').value * price_today

    # 計算并且將總收益顯示在 L20
    tsmc_sheet.cells(20, 'L').value = tsmc_sheet.cells(last_row, 'I').value - tsmc_sheet.cells(18, 'L').value

In [17]:
balance = 0

for sheet in wb.sheets:
    sheet.activate()
    run_back_test(sheet)
    balance += sheet.cells(20, 'L').value
    
print("投資組合收益： ${}".format(balance))

投資組合收益： $-47800.0
